In [6]:
import numpy as np
import os
from utils import check_paths

import mne
from mne.stats import permutation_cluster_1samp_test

import scipy
from scipy.stats import zscore
from scipy.sparse import coo_matrix, save_npz

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib qt

**Interpretaion of z-scored PAC stats:**
Negative z-scores don’t mean “negative PAC,” just “PAC lower than the group mean.”

**Significant cluster interpretaition:**
It is invalid to interpret the cluster p value as being spatially or temporally specific. A cluster with sufficiently low (for example < 0.05) p value at specific location does not allow you to say that the significant effect is at that particular location. The p value only tells you about the probability of obtaining similar or stronger/larger cluster anywhere in the data if there were no differences between the compared conditions. So it only allows you to draw conclusions about the differences in the data “in general”, not at specific locations.

**How NOT to interpret results from a cluster-based permutation test:**
https://www.fieldtriptoolbox.org/faq/stats/clusterstats_interpretation/

In [7]:
def plot_rect_topo_from_epochs(data, epochs_info, cmap='YlGn', vmin=None, vmax=None, title=''):
    """
    Plot a rectangular grid topomap from per-channel data using layout from epochs.info.

    Parameters
    ----------
    data : array, shape (n_channels,)
        Scalar values per channel (e.g., PAC strength).
    epochs_info : instance of mne.Info
        Info object from Epochs to extract channel positions.
    cmap : str or Colormap
        Colormap to use for values.
    vmin, vmax : float
        Limits for color scaling.
    title : str
        Title for the plot.
    """
    # Get rectangular layout from MNE
    layout = mne.channels.make_eeg_layout(epochs_info)
    ch_names = layout.names
    pos_2d = layout.pos[:, :2]

    # Normalize positions to grid indices
    x_idx = np.round((pos_2d[:, 0] - np.min(pos_2d[:, 0])) / np.ptp(pos_2d[:, 0]) * 14).astype(int)
    y_idx = np.round((pos_2d[:, 1] - np.min(pos_2d[:, 1])) / np.ptp(pos_2d[:, 1]) * 14).astype(int)
    layout_grid = {ch: (y, x) for ch, x, y in zip(ch_names, x_idx, y_idx)}
    # name, pos in zip(layout.names, layout.pos)

    # Prepare grid size
    nrows = y_idx.max() + 1
    ncols = x_idx.max() + 1

    # Start plotting
    fig, ax = plt.subplots(figsize=(ncols, nrows))
    ax.set_xlim(0, ncols)
    ax.set_ylim(0, nrows)
    ax.invert_yaxis()
    ax.set_xticks([])
    ax.set_yticks([])

    norm = plt.Normalize(vmin if vmin is not None else np.nanmin(data),
                         vmax if vmax is not None else np.nanmax(data))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

    for i, ch in enumerate(ch_names):
        # if ch not in layout_grid:
        #     continue
        row, col = layout_grid[ch]
        value = data[i].T
        color = sm.to_rgba(value)
        rect = patches.Rectangle((col, row), 1, 1, facecolor=color, edgecolor='black')
        ax.add_patch(rect)
        ax.text(col + 0.5, row + 0.5, ch, ha='center', va='center', fontsize=7)

    plt.gca().invert_yaxis()
    plt.colorbar(sm, ax=ax, shrink=0.8, label='Value')
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

    return fig, ax


In [8]:
def plot_matrix_topo_from_epochs(data, epochs_info, cmap='YlGn', vmin=None, vmax=None, title=''):
    """
    Plot a topographic layout of 2D matrices (e.g., PAC) per channel using epochs.info.

    Parameters
    ----------
    data : ndarray, shape (n_channels, height, width)
        A 2D matrix per channel (e.g., PAC frequency x frequency).
    epochs_info : instance of mne.Info
        Info object to extract channel layout.
    cmap : str or Colormap
        Colormap to use.
    vmin, vmax : float or None
        Color scale limits.
    title : str
        Title for the entire plot.
    """
    n_channels, h, w = data.shape

    # Get layout info
    layout = mne.channels.make_eeg_layout(epochs_info)
    ch_names = layout.names
    pos_2d = layout.pos[:, :2]

    # Normalize positions to grid indices
    x_idx = np.round((pos_2d[:, 0] - np.min(pos_2d[:, 0])) / np.ptp(pos_2d[:, 0]) * 14).astype(int)
    y_idx = np.round((pos_2d[:, 1] - np.min(pos_2d[:, 1])) / np.ptp(pos_2d[:, 1]) * 14).astype(int)
    layout_grid = {ch: (y, x) for ch, x, y in zip(ch_names, x_idx, y_idx)}

    # Grid dimensions
    nrows = y_idx.max() + 1
    ncols = x_idx.max() + 1

    # Set up figure
    fig, ax = plt.subplots(figsize=(ncols, nrows))
    ax.set_xlim(0, ncols)
    ax.set_ylim(0, nrows)
    # ax.invert_yaxis()
    ax.set_xticks([])
    ax.set_yticks([])

    # Color normalization
    norm = plt.Normalize(vmin if vmin is not None else np.nanmin(data),
                         vmax if vmax is not None else np.nanmax(data))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

    for i, ch in enumerate(ch_names):
        if ch not in layout_grid:
            continue
        row, col = layout_grid[ch]
        matrix = data[i].T

        # Plot small matrix inside rectangle
        extent = (col, col + 1, row, row + 1)
        ax.imshow(matrix, cmap=cmap, norm=norm, extent=extent, origin='lower', aspect='auto')

        # Draw frame and label
        rect = patches.Rectangle((col, row), 1, 1, fill=False, edgecolor='black', linewidth=0.5)
        ax.add_patch(rect)
        ax.text(col + 0.5, row + 0.5, ch, ha='center', va='center', fontsize=6, color='black')

    plt.colorbar(sm, ax=ax, shrink=0.7, label='Matrix Value')
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

    return fig, ax


In [9]:
def plot_significant_topomap(T_obs, clusters, cluster_p_values, info, p_thresh=0.05, vlim=(-4, 4),
                             group=None, task=None, task_stage=None, block_name=None):
    """
    Plots a topomap highlighting electrodes in significant clusters.

    Parameters
    ----------
    T_obs : array, shape (n_channels,)
        Observed T-values.
    clusters : list of boolean arrays
        Cluster masks (n_channels,).
    cluster_p_values : array
        P-values for each cluster.
    info : instance of mne.Info
        EEG info with channel locations.
    p_thresh : float
        Significance threshold.
    title : str
        Title for the plot.
    """
    # Combine significant cluster masks
    sig_mask = np.zeros_like(T_obs, dtype=bool)
    for cluster, p_val in zip(clusters, cluster_p_values):
        if p_val <= p_thresh:
            sig_mask |= cluster

    # Get color limits manually
    if vlim is None:
        vlim = np.nanmax(np.abs(T_obs))

    title = f'{group}{task}{task_stage}{block_name}: Significant Electrodes'
    # Start plotting
    fig, ax = plt.subplots()
    im, _ = mne.viz.plot_topomap(
        T_obs,
        info,
        cmap='PiYG',
        vlim=vlim,
        show=False,
        mask=sig_mask,
        mask_params=dict(marker='x', markersize=10, markerfacecolor='k'),
        contours=0,
        axes=ax
    )
    plt.colorbar(im, ax=ax, shrink=0.6, label='T-value')
    ax.set_title(title)
    plt.tight_layout()
    plt.show()


______________________

In [11]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'

groups = ['Y', 'O']
task = '_BL' # ['_BL', '_MAIN']
task_stages = ['_plan', '_go']

theta_range = np.around(np.linspace(4, 8, 20), 1)  # Phase: 4-8 Hz
gamma_range = np.around(np.linspace(30, 80, 20), 1)  # Amplitude: 30-80 Hz

In [ ]:
for group in groups:
    group_save_path = os.path.join(eeg_data_dir, f'{group} group')
    pac_stats_save_path = os.path.join(group_save_path, 'pac_stats')
    check_paths(pac_stats_save_path)

    subs = os.listdir(os.path.join(eeg_data_dir, group))

    for task_stage in task_stages:

        print(f'Processing {group} group, {task} task, {task_stage} stage...')

        ############# STACK PAC DATA OF INDIVIDUAL PARTICIPANTS #############

        # Create a list to store the PAC data for each subject
        pac_list = []
        pac_zscore_list = []

        for sub_name in subs:

            sub_dir = os.path.join(eeg_data_dir, group, sub_name)
            pac_dir = os.path.join(sub_dir, 'pac_results')
            
            # Get info about chnnals from one participant for adjacency matrix
            if sub_name == subs[0]: # read one epochs file to extract info
                # Load EEG data
                epochs_path = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
                epochs = mne.read_epochs(os.path.join(epochs_path, f"{sub_name}{task}_epochs{task_stage}-epo.fif"), preload=True)
                eeg_channel_names = epochs.copy().pick("eeg").ch_names
                epochs.pick(eeg_channel_names)
                # info = epochs.info
                # epochs.pick(choi)

            # Load PAC data
            pac = np.load(os.path.join(pac_dir, f"pac_mi_TOPO_{sub_name[-5:]}{task}{task_stage}.npy"))
            pac_t = np.transpose(pac, (1, 0, 2))
            pac_list.append(pac_t)
            pac_zscore_list.append(zscore(pac_t))
        
        # Stack them along a new first axis (subject axis)
        pac_all = np.stack(pac_list, axis=0)

        # Z-score the PAC data across subjects
        pac_zscore_all = np.stack(pac_zscore_list, axis=0)
        print(pac_all.shape) # (24, 60, 20, 20) subs x electrodes x ph_freqs x amp_freqs
        print(pac_zscore_all.shape)

        # Average z-scored PAC over subjects
        pac_zscore_mean = pac_zscore_all.mean(axis=0)  # shape: (60, 20, 20)

        # Save the PAC data
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_RAW.npy"), pac_all)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_ZSCORE.npy"), pac_zscore_all)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_ZSCORE_MEAN.npy"), pac_zscore_mean)

        ############### CREATE ADJACENCY MATRIX FOR STATISTICAL TEST #############
        # find_ch_adjacency first attempts to find an existing "neighbor"
        # (adjacency) file for given sensor layout.
        # If such a file doesn't exist, an adjacency matrix is computed on the fly,
        # using Delaunay triangulations.
        sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(epochs.info, "eeg")
        adjacency = mne.stats.combine_adjacency(
                                                sensor_adjacency,
                                                pac_zscore_all.shape[2],
                                                pac_zscore_all.shape[3]
                                                )
        print(adjacency.shape)

        # Save adjacency matrix
        adjacency_matrix = coo_matrix(adjacency)
        save_npz(f"{group}{task}{task_stage}_adjacency_matrix.npz", adjacency_matrix)

        ############# RUN CLUSTER-BASED PERMUTATION TEST #############

        tail = 0 # two-tailed test

        # Set the threshold for including data bins in clusters with t-value corresponding to p=0.01
        # Because we conduct a two-tailed test, we divide the p-value by 2 (which means we're making use of both tails of the distribution).
        # As the degrees of freedom, we specify the number of observations (here: subjects) minus 1.
        # Finally, we subtract 0.01 / 2 from 1, to get the critical t-value on the right tail
        degrees_of_freedom = pac_all.shape[0] - 1
        t_thresh = scipy.stats.t.ppf(1 - 0.01 / 2, df=degrees_of_freedom)

        # Set the number of permutations
        n_permutations = 10000

        # Run the analysis
        T_obs, clusters, cluster_p_values, H0 = permutation_cluster_1samp_test(
            pac_zscore_all,
            n_permutations=n_permutations,
            threshold=t_thresh,
            tail=tail,
            adjacency=adjacency,
            out_type="mask",
            max_step=1,
            n_jobs=-1,
            verbose=True,
        )

        # Save the results
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_T_obs.npy"), T_obs)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_clusters.npy"), np.array(clusters, dtype=object))
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_cluster_p_values.npy"), cluster_p_values)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_H0.npy"), H0)

        # SANITY CHECKS
        print(f't_thresh = {t_thresh}')
        print(f'T_obs_mean = {T_obs.mean()}')
        print(f'cluster_p_values = {cluster_p_values}')

        alpha = 0.05  # significance threshold
        significant_clusters = [i for i, p in enumerate(cluster_p_values) if p < alpha]
        print(f"Found {len(significant_clusters)} significant clusters")


        ####### PLOT THE RESULTS #######
        # Create directories for saving figures
        fig_group_path = os.path.join(pac_stats_save_path, 'figs')
        fig_group_save_path = os.path.join(fig_group_path, group)
        fig_task_save_path = os.path.join(fig_group_path, group, task)
        check_paths(fig_group_path, fig_group_save_path, fig_task_save_path)

        # Reshape the stats results
        sig_mask = np.zeros_like(T_obs, dtype=bool)

        for i in significant_clusters:
            sig_mask[clusters[i].reshape(pac_zscore_all.shape[1:])] = True


        for elec_idx in range(pac_zscore_mean.shape[0]):
            # Create masked array: T-values where significant, NaN elsewhere
            masked_T = np.where(sig_mask[elec_idx], T_obs[elec_idx], np.nan)

            # # Build a colormap with gray for NaNs
            cmap = plt.cm.PiYG.copy()  # try 'RdYlBu' or 'PRGn' (purple-green)
            cmap.set_bad(color='lightgray')  # this sets the NaNs to gray

            # Plot
            plt.figure(figsize=(6, 5))
            im = plt.imshow(
                masked_T.T,
                origin='lower',
                aspect='equal',
                cmap=cmap,
                interpolation='none',
                vmin=-10,
                vmax=10
            )
            plt.colorbar(im, label='T-value')
            plt.title(f'{group}{task}{task_stage}: PAC Cluster stats - {eeg_channel_names[elec_idx]}')
            plt.xticks(np.arange(len(theta_range)), theta_range, rotation='vertical')
            plt.yticks(np.arange(len(gamma_range)), gamma_range)
            plt.xlabel('Phase Freq Index')
            plt.ylabel('Amp Freq Index')
            plt.tight_layout()
            plt.show()

            plt.savefig(os.path.join(fig_task_save_path, f"pac_cluster_stats_{group}{task}{task_stage}_{eeg_channel_names[elec_idx]}.png"), dpi=300)


______________________________

**STATS FOR DATA AVERAGED ACROSS PHASE AND AMPLITUDE FREQUENCIES**

One condition

In [ ]:
# Main script to process PAC data and run cluster-based permutation tests
for group in groups:
    group_save_path = os.path.join(eeg_data_dir, f'{group} group')
    pac_stats_save_path = os.path.join(group_save_path, 'pac_stats')
    check_paths(pac_stats_save_path)
    
    # Create directories for saving figures
    fig_group_path = os.path.join(pac_stats_save_path, 'figs')
    fig_group_save_path = os.path.join(fig_group_path, group)
    fig_task_save_path = os.path.join(fig_group_path, group, task)

    subs = os.listdir(os.path.join(eeg_data_dir, group))

    for task_stage in task_stages:

        print(f'Processing {group} group, {task} task, {task_stage} stage,  block...')

        ############# STACK PAC DATA OF INDIVIDUAL PARTICIPANTS #############

        # Create a list to store the PAC data for each subject
        pac_list = []
        pac_zscore_list = []

        for sub_name in subs:

            sub_dir = os.path.join(eeg_data_dir, group, sub_name)
            pac_dir = os.path.join(sub_dir, 'pac_results')
            
            # Get info about chnnals from one participant for adjacency matrix
            if sub_name == subs[0]: # read one epochs file to extract info
                # Load EEG data
                epochs_path = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
                epochs = mne.read_epochs(os.path.join(epochs_path, f"{sub_name}{task}_epochs{task_stage}-epo.fif"), preload=True)
                eeg_channel_names = epochs.copy().pick("eeg").ch_names
                epochs.pick(eeg_channel_names)

            # Load PAC data
            pac = np.load(os.path.join(pac_dir, f"pac_mi_TOPO_{sub_name[-5:]}{task}{task_stage}.npy"))
            pac_t = np.transpose(pac, (1, 0, 2))
            pac_list.append(pac_t)
            pac_zscore_list.append(zscore(pac_t))

        # Stack them along a new first axis (subject axis)
        pac_all = np.stack(pac_list, axis=0)

        # Z-score the PAC data across subjects
        pac_zscore_all = np.stack(pac_zscore_list, axis=0)
        print('PAC array shape:', pac_all.shape) # (24, 60, 20, 20) subs x electrodes x ph_freqs x amp_freqs
        print('z-scored PAC array shape:', pac_zscore_all.shape)

        # Averafe z-scored PAC over phase and amplitude frequencies
        pac_zscore_all_ave = np.mean(pac_zscore_all, axis=(2, 3)) # (24, 60) subs x electrodes
        # pac_zscore_all_med = np.median(pac_zscore_all, axis=(2, 3)) # produces more significant clusters

        # # Save the PAC data
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_ZSCORE_freqs_ave.npy"), pac_zscore_all_ave)

        # ############# PLOT AND SAVE Z-SCORED PAC AVERAGED ACROSS PARTICIPANTS #############
        pac_plot, ax1 = plot_rect_topo_from_epochs(np.mean(pac_zscore_all_ave, axis=(0)), epochs.info,
                                                title=f'{group}{task}{task_stage}: Averaged z-scored PAC MI',
                                                cmap='PiYG', vmin=-0.5, vmax=0.5)
        plt.savefig(os.path.join(fig_task_save_path, f"pac_mi_{group}{task}{task_stage}_PAC_MI_AVE_TOPO.png"), dpi=300)

        pac_ave_plot, ax2 = plot_matrix_topo_from_epochs(np.mean(pac_zscore_all, axis=(0)), epochs.info,
                                                        title=f'{group}{task}{task_stage}: z-scored PAC MI',
                                                        cmap='PiYG', vmin=-0.5, vmax=0.5)
        plt.savefig(os.path.join(fig_task_save_path, f"pac_mi_{group}{task}{task_stage}_PAC_MI_TOPO.png"), dpi=300)

        ############### CREATE ADJACENCY MATRIX FOR STATISTICAL TEST #############
        # find_ch_adjacency first attempts to find an existing "neighbor"
        # (adjacency) file for given sensor layout.
        # If such a file doesn't exist, an adjacency matrix is computed on the fly,
        # using Delaunay triangulations.
        sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(epochs.info, "eeg")
        print('adjacency shape:', sensor_adjacency.shape)


        ############# RUN CLUSTER-BASED PERMUTATION TEST #############

        tail = 0 # two-tailed test

        # Set the threshold for including data bins in clusters with t-value corresponding to p=0.01
        # Because we conduct a two-tailed test, we divide the p-value by 2 (which means we're making use of both tails of the distribution).
        # As the degrees of freedom, we specify the number of observations (here: subjects) minus 1.
        # Finally, we subtract 0.01 / 2 from 1, to get the critical t-value on the right tail
        degrees_of_freedom = pac_all.shape[0] - 1
        t_thresh = scipy.stats.t.ppf(1 - 0.01 / 2, df=degrees_of_freedom)

        #!
        # threshold_tfce = dict(start=0, step=0.2) # Threshold-free cluster enhancement (TFCE) - more conservative, similar results

        # Set the number of permutations
        n_permutations = 10000

        # Run the analysis
        T_obs, clusters, cluster_p_values, H0 = permutation_cluster_1samp_test(
            pac_zscore_all_ave,
            n_permutations=n_permutations,
            threshold=t_thresh,
            tail=tail,
            adjacency=sensor_adjacency,
            out_type="mask",
            max_step=1,
            verbose=True,
        )

        # # Save the results
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_T_obs.npy"), T_obs)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_clusters.npy"), np.array(clusters, dtype=object))
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_cluster_p_values.npy"), cluster_p_values)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_H0.npy"), H0)

        # SANITY CHECKS
        print(f't_thresh = {t_thresh}')
        print(f'T_obs_mean = {T_obs.mean()}')
        print(f'cluster_p_values = {cluster_p_values}')

        alpha = 0.05  # significance threshold
        significant_clusters = [i for i, p in enumerate(cluster_p_values) if p < alpha]
        print(f"Found {len(significant_clusters)} significant clusters")


        ####### PLOT THE RESULTS #######
        plot_significant_topomap(T_obs, clusters, cluster_p_values, epochs.info,
                                 group=group, task=task, task_stage=task_stage, block_name='')
        plt.savefig(os.path.join(fig_task_save_path, f"pac_cluster_stats_{group}{task}{task_stage}_freq_ave_TOPO.png"), dpi=300)

STOPPED HERE

ADD COMPARISON ACROSS CONDITIONS

In [ ]:
# Main script to process PAC data and run cluster-based permutation tests
for group in groups:
    group_save_path = os.path.join(eeg_data_dir, f'{group} group')
    pac_stats_save_path = os.path.join(group_save_path, 'pac_stats')
    check_paths(pac_stats_save_path)
    
    # Create directories for saving figures
    fig_group_path = os.path.join(pac_stats_save_path, 'figs')
    fig_group_save_path = os.path.join(fig_group_path, group)
    fig_task_save_path = os.path.join(fig_group_path, group, task)

    subs = os.listdir(os.path.join(eeg_data_dir, group))

    for task_stage in task_stages:

        print(f'Processing {group} group, {task} task, {task_stage} stage,  block...')

        ############# STACK PAC DATA OF INDIVIDUAL PARTICIPANTS #############

        # Create a list to store the PAC data for each subject
        pac_list = []
        pac_zscore_list = []

        for sub_name in subs:

            sub_dir = os.path.join(eeg_data_dir, group, sub_name)
            pac_dir = os.path.join(sub_dir, 'pac_results')
            
            # Get info about chnnals from one participant for adjacency matrix
            if sub_name == subs[0]: # read one epochs file to extract info
                # Load EEG data
                epochs_path = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
                epochs = mne.read_epochs(os.path.join(epochs_path, f"{sub_name}{task}_epochs{task_stage}-epo.fif"), preload=True)
                eeg_channel_names = epochs.copy().pick("eeg").ch_names
                epochs.pick(eeg_channel_names)

            # Load PAC data
            pac = np.load(os.path.join(pac_dir, f"pac_mi_TOPO_{sub_name[-5:]}{task}{task_stage}.npy"))
            pac_t = np.transpose(pac, (1, 0, 2))
            pac_list.append(pac_t)
            pac_zscore_list.append(zscore(pac_t))

        # Stack them along a new first axis (subject axis)
        pac_all = np.stack(pac_list, axis=0)

        # Z-score the PAC data across subjects
        pac_zscore_all = np.stack(pac_zscore_list, axis=0)
        print('PAC array shape:', pac_all.shape) # (24, 60, 20, 20) subs x electrodes x ph_freqs x amp_freqs
        print('z-scored PAC array shape:', pac_zscore_all.shape)

        # Averafe z-scored PAC over phase and amplitude frequencies
        pac_zscore_all_ave = np.mean(pac_zscore_all, axis=(2, 3)) # (24, 60) subs x electrodes
        # pac_zscore_all_med = np.median(pac_zscore_all, axis=(2, 3)) # produces more significant clusters

        # # Save the PAC data
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_ZSCORE_freqs_ave.npy"), pac_zscore_all_ave)

        # ############# PLOT AND SAVE Z-SCORED PAC AVERAGED ACROSS PARTICIPANTS #############
        pac_plot, ax1 = plot_rect_topo_from_epochs(np.mean(pac_zscore_all_ave, axis=(0)), epochs.info,
                                                title=f'{group}{task}{task_stage}: Averaged z-scored PAC MI',
                                                cmap='PiYG', vmin=-0.5, vmax=0.5)
        plt.savefig(os.path.join(fig_task_save_path, f"pac_mi_{group}{task}{task_stage}_PAC_MI_AVE_TOPO.png"), dpi=300)

        pac_ave_plot, ax2 = plot_matrix_topo_from_epochs(np.mean(pac_zscore_all, axis=(0)), epochs.info,
                                                        title=f'{group}{task}{task_stage}: z-scored PAC MI',
                                                        cmap='PiYG', vmin=-0.5, vmax=0.5)
        plt.savefig(os.path.join(fig_task_save_path, f"pac_mi_{group}{task}{task_stage}_PAC_MI_TOPO.png"), dpi=300)

        ############### CREATE ADJACENCY MATRIX FOR STATISTICAL TEST #############
        # find_ch_adjacency first attempts to find an existing "neighbor"
        # (adjacency) file for given sensor layout.
        # If such a file doesn't exist, an adjacency matrix is computed on the fly,
        # using Delaunay triangulations.
        sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(epochs.info, "eeg")
        print('adjacency shape:', sensor_adjacency.shape)


        ############# RUN CLUSTER-BASED PERMUTATION TEST #############

        tail = 0 # two-tailed test

        # Set the threshold for including data bins in clusters with t-value corresponding to p=0.01
        # Because we conduct a two-tailed test, we divide the p-value by 2 (which means we're making use of both tails of the distribution).
        # As the degrees of freedom, we specify the number of observations (here: subjects) minus 1.
        # Finally, we subtract 0.01 / 2 from 1, to get the critical t-value on the right tail
        degrees_of_freedom = pac_all.shape[0] - 1
        t_thresh = scipy.stats.t.ppf(1 - 0.01 / 2, df=degrees_of_freedom)

        #!
        # threshold_tfce = dict(start=0, step=0.2) # Threshold-free cluster enhancement (TFCE) - more conservative, similar results

        # Set the number of permutations
        n_permutations = 10000

        # Run the analysis
        T_obs, clusters, cluster_p_values, H0 = permutation_cluster_1samp_test(
            pac_zscore_all_ave,
            n_permutations=n_permutations,
            threshold=t_thresh,
            tail=tail,
            adjacency=sensor_adjacency,
            out_type="mask",
            max_step=1,
            verbose=True,
        )

        # # Save the results
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_T_obs.npy"), T_obs)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_clusters.npy"), np.array(clusters, dtype=object))
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_cluster_p_values.npy"), cluster_p_values)
        np.save(os.path.join(pac_stats_save_path, f"pac_mi_{group}{task}{task_stage}_freqs_ave_H0.npy"), H0)

        # SANITY CHECKS
        print(f't_thresh = {t_thresh}')
        print(f'T_obs_mean = {T_obs.mean()}')
        print(f'cluster_p_values = {cluster_p_values}')

        alpha = 0.05  # significance threshold
        significant_clusters = [i for i, p in enumerate(cluster_p_values) if p < alpha]
        print(f"Found {len(significant_clusters)} significant clusters")


        ####### PLOT THE RESULTS #######
        plot_significant_topomap(T_obs, clusters, cluster_p_values, epochs.info,
                                 group=group, task=task, task_stage=task_stage, block_name='')
        plt.savefig(os.path.join(fig_task_save_path, f"pac_cluster_stats_{group}{task}{task_stage}_freq_ave_TOPO.png"), dpi=300)

_________________________________